# 🎬 VideoRobot — One-Click Colab Runner (Fixed)\nاین نوت‌بوک ریپو را کلون می‌کند، وابستگی‌ها و FFmpeg را نصب می‌کند، بک‌اند را **به‌صورت پکیج** بالا می‌آورد، سلامت را چک می‌کند، در صورت وجود دارایی‌ها از Google Drive استفاده می‌کند، و یک رندر نمونه می‌گیرد.

In [ ]:
# 🧠 Mount Google Drive (optional)\ntry:\n    from google.colab import drive\n    drive.mount('/content/drive')\n    print('✅ Google Drive mounted. Optional assets path: /content/drive/MyDrive/videorobot_assets')\nexcept Exception as e:\n    print('ℹ️ Drive not mounted (non-Colab environment)')

In [ ]:
# 📦 Clone repository fresh\nimport shutil, pathlib, subprocess\nREPO_URL = "https://github.com/englishpodcasteasy-glitch/videorobot.git"\nWORK = pathlib.Path('/content/videorobot')\nshutil.rmtree(WORK, ignore_errors=True)\nsubprocess.run(['git', 'clone', REPO_URL, str(WORK)], check=True)\nprint('✅ Repo cloned at', WORK)

In [ ]:
# ⚙️ Install FFmpeg and Python deps\n!apt-get update -qq && apt-get install -y ffmpeg\n!pip install -q -r /content/videorobot/backend/requirements.txt

In [ ]:
# 🩹 Ensure backend is a proper package and fix absolute imports if any\nimport re, pathlib\nBACKEND = pathlib.Path('/content/videorobot/backend')\n(BACKEND / '__init__.py').write_text('')\npatterns = [\n  (r"\\bfrom\\s+config\\s+import\\b",            "from .config import"),\n  (r"\\bfrom\\s+scheduler\\s+import\\b",         "from .scheduler import"),\n  (r"\\bfrom\\s+utils\\s+import\\b",             "from .utils import"),\n  (r"\\bfrom\\s+renderer_service\\s+import\\b",  "from .renderer_service import"),\n  (r"\\bfrom\\s+renderer\\s+import\\b",          "from .renderer import"),\n  (r"(^|\\n)\\s*import\\s+config(\\s|$)",        r"\\1from . import config\\2"),\n  (r"(^|\\n)\\s*import\\s+utils(\\s|$)",         r"\\1from . import utils\\2")\n]\nchanged = []\nfor py in BACKEND.rglob('*.py'):\n    txt = py.read_text(encoding='utf-8', errors='ignore')\n    orig = txt\n    for pat, rep in patterns:\n        txt = re.sub(pat, rep, txt)\n    if txt != orig:\n        py.write_text(txt, encoding='utf-8')\n        changed.append(py.name)\nprint('🛠️ Patched:', changed if changed else '(no changes needed)')

In [ ]:
# 🚀 Start backend as PACKAGE and wait for health\nimport os, sys, time, subprocess, requests, socket, pathlib\nREPO = '/content/videorobot'\nLOG = '/content/backend.log'\ndef pick_port():\n    for p in (int(os.environ.get('BACKEND_PORT','8000')), 8001, 8002):\n        s=socket.socket()\n        try: s.bind(('127.0.0.1', p)); s.close(); return p\n        except OSError: continue\n    return 8000\nPORT = pick_port(); os.environ['BACKEND_PORT'] = str(PORT)\nprint('🚪 BACKEND_PORT =', PORT)\nenv = os.environ.copy(); env['PYTHONUNBUFFERED']='1'\nlog_f = open(LOG, 'w', encoding='utf-8')\nproc = subprocess.Popen([sys.executable, '-m', 'backend.main'], cwd=REPO,\n                        stdout=log_f, stderr=subprocess.STDOUT, text=True, env=env)\nprint('PID:', proc.pid, 'LOG:', LOG)\nBASE = f"http://127.0.0.1:{PORT}"\nok = False\nfor _ in range(180):\n    for path in ('/healthz','/health','/version'):\n        try:\n            r = requests.get(BASE+path, timeout=2)\n            if r.ok:\n                print('✅ Health via', path, '→', r.text)\n                ok = True\n                break\n        except Exception:\n            pass\n    if ok: break\n    time.sleep(1)\nif not ok:\n    log_f.flush(); log_f.close()\n    import subprocess as sp\n    print('\n❌ Backend not healthy. Tail log:\n' + '-'*60)\n    print(sp.getoutput(f'tail -n 200 {LOG} || true'))\n    raise SystemExit('Backend failed to become healthy.')\nlog_f.flush(); log_f.close()\nprint('\n✅ Backend ready at', BASE)

In [ ]:
# 📝 Create a simple manifest; optionally use Drive assets if present\nimport json, pathlib\nASSETS = pathlib.Path('/content/drive/MyDrive/videorobot_assets')\nmanifest = {\n  "seed": 7,\n  "video": {"width": 720, "height": 1280, "fps": 30, "bg_color": "#101318"},\n  "tracks": [\n    {"type": "text", "content": "سلام دنیا", "start": 0.2, "duration": 3, "x": 40, "y": 80, "size": 72, "color": "#FFFFFF"}\n  ]\n}\ntry:\n    vids = list(ASSETS.glob('*.mp4'))\n    imgs = list(ASSETS.glob('*.png')) + list(ASSETS.glob('*.jpg'))\n    auds = list(ASSETS.glob('*.mp3')) + list(ASSETS.glob('*.wav'))\n    if vids:\n        manifest['tracks'].insert(0, {"type":"video", "src": str(vids[0]), "start":0, "fit":"cover"})\n    if imgs:\n        manifest['tracks'].insert(0, {"type":"image", "src": str(imgs[0]), "start":0, "duration": 5, "x":0, "y":0, "scale":1.0})\n    if auds:\n        manifest['tracks'].append({"type":"audio", "src": str(auds[0]), "start":0, "gain_db": -3})\nexcept Exception:\n    pass\nMANIFEST_PATH = pathlib.Path('/content/manifest.json')\nMANIFEST_PATH.write_text(json.dumps(manifest, ensure_ascii=False, indent=2))\nprint('✅ Manifest at', MANIFEST_PATH)

In [ ]:
# 🎥 Submit render; retry once on MoviePy hint; poll; download MP4\nimport os, json, time, requests, urllib.request, pathlib, sys, subprocess\nBASE = f"http://127.0.0.1:{os.environ.get('BACKEND_PORT','8000')}"\ndef maybe_fix(text):\n    t=(text or '').lower()\n    if any(k in t for k in ['moviepy','imageio-ffmpeg','videofileclip','ffmpeg not found']):\n        print('⚠️ MoviePy/FFmpeg missing per server. Installing...')\n        subprocess.run([sys.executable,'-m','pip','install','-q','moviepy>=1.0.3,<2','imageio-ffmpeg>=0.4.9,<1'], check=False)\n        return True\n    return False\npayload = json.loads(open('/content/manifest.json','r',encoding='utf-8').read())\ndef submit(payload):\n    r = requests.post(BASE+'/render', json=payload, timeout=120)\n    print('POST /render:', r.status_code)\n    try: print(json.dumps(r.json(), ensure_ascii=False, indent=2))\n    except Exception: print(r.text[:400])\n    if not r.ok:\n        body = ''\n        try: body = r.json().get('error') or r.text\n        except Exception: body = r.text\n        if maybe_fix(body):\n            time.sleep(2)\n            r = requests.post(BASE+'/render', json=payload, timeout=120)\n            print('POST /render (retry):', r.status_code)\n            try: print(json.dumps(r.json(), ensure_ascii=False, indent=2))\n            except Exception: print(r.text[:400])\n    r.raise_for_status()\n    js = r.json()\n    return js.get('job_id') or (js.get('data') or {}).get('job_id')\njob_id = submit(payload)\nassert job_id, '❌ No job_id returned'\nprint('🎬 job_id:', job_id)\nfor i in range(900):\n    g = requests.get(BASE+f'/progress/{job_id}', timeout=10)\n    if g.ok:\n        js = g.json()\n        state = js.get('state') or (js.get('data') or {}).get('state')\n        pct = js.get('pct') or (js.get('data') or {}).get('pct')\n        msg = js.get('message') or (js.get('data') or {}).get('message')\n        print(f'tick {i}:', state, pct, msg or '')\n        if state == 'success': break\n        if state == 'error': raise SystemExit(f"❌ render failed: {js}")\n    else:\n        print('progress http', g.status_code, g.text[:200])\n    time.sleep(1)\nOUT = f"/content/{job_id}.mp4"\ntry:\n    urllib.request.urlretrieve(BASE+f'/download?jobId={job_id}', OUT)\n    print('📦 Downloaded:', OUT)\nexcept Exception as e:\n    # fallback to known path layout\n    cand = pathlib.Path(f'/content/outputs/{job_id}/final.mp4')\n    if cand.exists():\n        print('📦 Found:', str(cand))\n    else:\n        raise